In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline


In [2]:
# df = pd.read_csv("../dataset/hr_train.csv")
# df.head()
# df.info()
# df.sales.unique()
# df.salary.unique()
# df.groupby(['salary', 'sales']).size()

# pd.get_dummies(df.salary)
# df = df.join(pd.get_dummies(df.salary))
# del df['salary']

# pd.get_dummies(df.sales)  # 读日编码？？？
# df = df.rename(columns={'sales':'department'})
# df = df.join(pd.get_dummies(df.department))
# del df['department']
# df.to_csv("../dataset/hr_train_ok.csv", index=False)

In [5]:
df = pd.read_csv("../dataset/hr_train_ok.csv")
del df['id']

In [6]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     11999 non-null  float64
 1   last_evaluation        11999 non-null  float64
 2   number_project         11999 non-null  int64  
 3   average_montly_hours   11999 non-null  int64  
 4   time_spend_company     11999 non-null  int64  
 5   Work_accident          11999 non-null  int64  
 6   left                   11999 non-null  int64  
 7   promotion_last_5years  11999 non-null  int64  
 8   high                   11999 non-null  int64  
 9   low                    11999 non-null  int64  
 10  medium                 11999 non-null  int64  
 11  IT                     11999 non-null  int64  
 12  RandD                  11999 non-null  int64  
 13  accounting             11999 non-null  int64  
 14  hr                     11999 non-null  int64  
 15  ma

In [7]:
# df.left.unique()
df.left.value_counts()
# len(df)  # df.size

0    9129
1    2870
Name: left, dtype: int64

In [8]:
df.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'high', 'low', 'medium', 'IT', 'RandD',
       'accounting', 'hr', 'management', 'marketing', 'product_mng', 'sales',
       'support', 'technical'],
      dtype='object')

In [9]:
Y_data = df.left.values.reshape(-1,1)  # pd type
# Y_data.shape
Y = torch.from_numpy(Y_data).type(torch.float32)
# Y = torch.from_numpy(Y_data).type(torch.FloatTensor)
Y.size()

torch.Size([11999, 1])

In [10]:
X_columns = [c for c in df.columns if c != 'left']
X_data = df[X_columns]  
X = torch.from_numpy(X_data.values).type(torch.float32)
# X = torch.from_numpy(X_data.values).type(torch.FloatTensor)
X.size(0)

11999

In [11]:
X_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     11999 non-null  float64
 1   last_evaluation        11999 non-null  float64
 2   number_project         11999 non-null  int64  
 3   average_montly_hours   11999 non-null  int64  
 4   time_spend_company     11999 non-null  int64  
 5   Work_accident          11999 non-null  int64  
 6   promotion_last_5years  11999 non-null  int64  
 7   high                   11999 non-null  int64  
 8   low                    11999 non-null  int64  
 9   medium                 11999 non-null  int64  
 10  IT                     11999 non-null  int64  
 11  RandD                  11999 non-null  int64  
 12  accounting             11999 non-null  int64  
 13  hr                     11999 non-null  int64  
 14  management             11999 non-null  int64  
 15  ma

In [12]:
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用

class Model(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(20, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.sigmoid( self.liner_3(x) )
        return x

# 更加高级语言法，等价于Model
class Model_high(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(20, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.Liner_1(input)
        x = self.relu(x)
        x = self.Liner_2(x)
        x = self.relu(x)
        x = self.Liner_3(x)
        x = self.sigmoid(x)
        return x



In [13]:
lr = 0.0001
def get_model():
    model = Model()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    return model, opt

In [14]:
model, optim = get_model()
# print( model, "\n" )
# print( optim )

Model(
  (liner_1): Linear(in_features=20, out_features=64, bias=True)
  (liner_2): Linear(in_features=64, out_features=64, bias=True)
  (liner_3): Linear(in_features=64, out_features=1, bias=True)
) 

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)


In [15]:
loss_fn = nn.BCELoss()
batch = 64
no_of_batches = len(X)//batch
epochs = 10
from torch.utils.data import TensorDataset, DataLoader

epochMode = 4

In [16]:
# Ver 01 - OK
if epochMode == 1:
    for epoch in range( epochs ):
        for i in range( no_of_batches):
            start = i*batch
            end = start + batch
            x = X[start: end]
            y = Y[start: end]
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
        with torch.no_grad():
            print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )




In [17]:
# Ver 02 - TensorDataset 整合x和y张量，简化
if epochMode == 2:
    HR_dataset = TensorDataset(X, Y)
    # len(HR_dataset)
    # HR_dataset[0:2]
    for epoch in range( epochs ):
        for i in range( no_of_batches):
            x, y = HR_dataset[i*batch: i*batch+batch]
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
        with torch.no_grad():
            print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

In [18]:
# Ver 03 - DadaLoader 进一步简化数据分批和乱序执行
epochMode = 3
if epochMode == 3:
    HR_dataset = TensorDataset(X, Y)
    HR_dataloader = DataLoader(HR_dataset, batch_size=batch, shuffle=True)  # shuffle乱序取数
    for epoch in range( epochs ):
        for x,y in HR_dataloader:
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
        with torch.no_grad():
            print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

epoch:  0  loss:  0.564719557762146
epoch:  1  loss:  0.5621052980422974
epoch:  2  loss:  0.5624903440475464
epoch:  3  loss:  0.5613702535629272
epoch:  4  loss:  0.5584651827812195
epoch:  5  loss:  0.5574606657028198
epoch:  6  loss:  0.5566904544830322
epoch:  7  loss:  0.5539653301239014
epoch:  8  loss:  0.5526372790336609
epoch:  9  loss:  0.5536019206047058


In [19]:
# Ver 4.1 - random_split Mode， charlie
from torch.utils.data import random_split    
def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).type(torch.int32)
    acc = (y_pred == y_true).float().mean()
    return acc

HR_dataset = TensorDataset(X, Y)
test_ds, test_ds = random_split(HR_dataset, [0.8, 0.2]) 
HR_dataloader = DataLoader(test_ds, batch_size=batch, shuffle=True)  # shuffle乱序取数
epochMode = 41
if epochMode == 41:  
    for epoch in range( epochs ):
        # for x,y in HR_dataloader:
        for x,y in HR_dataloader:
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
        with torch.no_grad():
            print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

epoch:  0  loss:  0.5540829300880432
epoch:  1  loss:  0.55328768491745
epoch:  2  loss:  0.5488249659538269
epoch:  3  loss:  0.550838828086853
epoch:  4  loss:  0.5481733083724976
epoch:  5  loss:  0.551487147808075
epoch:  6  loss:  0.5469626784324646
epoch:  7  loss:  0.5465898513793945
epoch:  8  loss:  0.5461639761924744
epoch:  9  loss:  0.5485742092132568


In [20]:
# Ver 4.2 - sklearn mode
def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).type(torch.int32)
    acc = (y_pred == y_true).float().mean()
    return acc

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

train_x = torch.from_numpy(train_x.values).type(torch.float32)
train_y = torch.from_numpy(train_y).type(torch.float32)
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

test_x = torch.from_numpy(test_x.values).type(torch.float32)
test_y = torch.from_numpy(test_y).type(torch.float32)
test_ds = TensorDataset(test_x, test_y)
test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量

In [21]:

# train_x.shape, test_x.shape
# train_x, test_y
# train_x.data

In [22]:

epochMode = 42
if epochMode == 42:  
    for epoch in range( epochs ):
        # for x,y in HR_dataloader:
        for x,y in train_dl:
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
        with torch.no_grad():
            print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )
model

epoch:  0  loss:  0.5437403917312622
epoch:  1  loss:  0.5518386960029602
epoch:  2  loss:  0.5394802093505859
epoch:  3  loss:  0.5378124117851257
epoch:  4  loss:  0.5424845218658447
epoch:  5  loss:  0.5324317216873169
epoch:  6  loss:  0.5294622778892517
epoch:  7  loss:  0.5325123071670532
epoch:  8  loss:  0.5235079526901245
epoch:  9  loss:  0.5209560990333557


Model(
  (liner_1): Linear(in_features=20, out_features=64, bias=True)
  (liner_2): Linear(in_features=64, out_features=64, bias=True)
  (liner_3): Linear(in_features=64, out_features=1, bias=True)
)

In [23]:
print( loss_fn(model(train_x), train_y) )
# print( loss_fn(model(train_x), train_y).data.item() )

tensor(0.5196, grad_fn=<BinaryCrossEntropyBackward0>)


In [24]:
print( loss_fn(model(train_x), train_y).data.item() )
accuracyRate = ((model(train_x).data.numpy()).astype('int') == train_y.numpy().astype('int')).mean()
print("训练数据 - 准确率%：", accuracyRate*100) 

0.5195857286453247
训练数据 - 准确率%： 76.13068118679853


In [25]:
print( loss_fn(model(test_x), test_y).data.item() )
accuracyRate = ((model(test_x).data.numpy()).astype('int') == test_y.numpy().astype('int')).mean()
print("测试数据 - 准确率%：", accuracyRate*100) 

0.5250666737556458
测试数据 - 准确率%： 75.93333333333334


- 数据准确性验证
- 过拟合/欠拟合
    - 过拟合：对于已知数据过度拟合，对未知数据预测性差
    - 欠拟合：对于已知数据拟合不够，对未知数据预测性差

In [26]:
# !conda list
# from sklearn.model_selection import train_test_split
# train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)
# train_x = torch.from_numpy(train_x).type(torch.float32)
# test_x = torch.from_numpy(test_x).type(torch.float32)
# train_y = torch.from_numpy(train_y).type(torch.float32)
# test_y = torch.from_numpy(test_y).type(torch.float32)

In [27]:
# (y_pred == y).float32